This is an IPython notebook. Each cell has code which can be run independently from what is run in other cells. Once a cell is
run the variables etc are kept and saved and not destroyed from the memory.

In [ ]:
print a

In [ ]:
a = 1

# now go back and the run the first cell. 
# To run a cell, click inside the cell and press Shift + Enter

### Session 1 - Basic visualisations and EDA

In [ ]:
# CHECKPOINT #1 - Getting started

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
sns.set_style('darkgrid')
%matplotlib inline

In [ ]:
df = pd.read_csv('train_csv.csv')
df.reset_index(drop=True, inplace=True)
df.head(5)

In [ ]:
## To get help related to any Python method, module etc. Write the module or methods name on a new line followed by a ? and
## execute.
df?



In [ ]:
## To know about methods in a cell. Write df. and press Tab. 
## It was show the list of methods inside df.
## To get help for the method, type df.method_name?

df.update?

In [ ]:
## What all columns got loaded?
df.info()

In [ ]:
## For now I am going to ignore Computer Programming, Electrical and etc scores as they are any reflected in the domain scores.
## Also removing DOJ etc.
## Removing ID
## Removing college city and job city for basic model.

basic_cols = [u'Salary', u'Gender', u'10percentage', u'10board', 
          u'12graduation', u'12percentage', u'12board', u'CollegeID', 
          u'CollegeTier', u'Degree', u'Specialization', u'collegeGPA', u'CollegeCityTier', u'CollegeState', 
          u'GraduationYear', u'English', u'Logical', u'Quant', u'Domain', u'conscientiousness', 
          u'agreeableness', u'extraversion', u'nueroticism', u'openess_to_experience', 'Designation']


In [ ]:
data = df[basic_cols]

In [ ]:
# CHECKPOINT #2 - Data inspection

In [ ]:
## Analysis of the numerical columns.
data.describe()

In [ ]:
# CHECKPOINT #3 - Inspecting further

In [ ]:
## Lets see the top 5 values in each columns - this will help get a feel of the non-numeric columns too like city etc.
## Which are are non-numerical columns? All columns minus numerical columns
non_num_cols = list(set(data.columns) - set(data._get_numeric_data().columns))

for col in non_num_cols:
    print col,'has unique values:' , data[col].unique().shape[0]
    print data[col].value_counts().iloc[:5]
    print '\n'

In [ ]:
### Cleaning CGPA 
data.collegeGPA.describe()

In [ ]:
data.collegeGPA.hist(bins=10)
plt.title('College GPA - Histogram')

In [ ]:
#We can notice there a few people who are below ten. It seems like an anamoly. There aren't a lot of students in betweem that
# bar and the 40 to 60 bar. It could be that the dataset has CGPA which also belong the the 0-10 scale and students didn't 
# scale it to 100. Different of scales has an issue as a the model would think a 9.98 CGPA is extremely bad in the current
# scale but it possibly was a good cgpa. We will scale the grades below the 10 scale to a 100 scale.

data.loc[data.collegeGPA <=10, 'collegeGPA'] = data.collegeGPA*10

In [ ]:
data.collegeGPA.hist()
plt.xlim(0,100)

In [ ]:
data.Domain.hist()

In [ ]:
## Setting the data which was -1 as null object.
for col in df.columns:
    df.loc[df[col] == -1, col] = np.nan


In [ ]:
## So we want to predict Salary
## But, how is the salary distibuted

data.Salary.hist(xrot =90, bins = 100)
plt.title('Salary - Histogram')

## You can notice longtail. What is longtail?

In [ ]:
## How does 12 board look like?
data['12board'].value_counts().plot(kind='bar', figsize=(16,5))
## 12 board follows a long tail

In [ ]:
## How does 12 board look like for top 30 values?
data['12board'].value_counts()[:30].plot(kind='bar', figsize=(16,5))
## You can see the main primary boards have peaks and then there is a long tail of boards which are not always unique.
## eg. up, upboard, up board and uttar pradesh board all mean the same thing but are represented differently.
## We could have cleaned this data using some scripts and added this column to our analysis but for now we will not use this.

In [ ]:
## How do English scores of candidates look?
data.English.hist(bins=20)
plt.title('English - Histogram')


In [ ]:
## But how does English affect Salary?
## What scores of English get what kinds of Salaries?
## Do higher english scores get you higher salaries?
## Is it after any particular score, that it starts to stop mattering?
sns.set_style('darkgrid')
plt.scatter(data.English, data.Salary)
plt.title('English v/s Salary')
plt.xlabel('English')
plt.ylabel('Salary')



In [ ]:
# Similarly for Logical Ability
# Can you fill in the missing code?
plt.scatter(    ,   )
plt.title('Logical v/s Salary')

In [ ]:
## Is Gender playing a role in deciding Salary? How do you visualise this relationship  between a category (Gender)
## and Salary?

In [ ]:
data.boxplot(by='Gender', column='Salary')
plt.ylim(0,800000)
## Note read line in median.
## How to read this plot?

In [ ]:
############ Play area for session one ############
## Why don't you plot a few scatters or hists etc for different variables?
data[''].hist()


In [ ]:
plt.scatter(data[''], data[''])

###  Session 2  - Let's make our first model

In [ ]:
# CHECKPOINT #4 - Modeling your data

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
# We are interested in predicting Salary
data.columns
# For now, we will try to predict using only these columns:
X_col = [u'Gender', u'10percentage', u'12graduation', u'12percentage', u'CollegeTier',
     u'Degree', u'collegeGPA', u'CollegeCityTier', u'GraduationYear', u'English', u'Logical', 
     u'Quant', u'Domain', u'conscientiousness', u'agreeableness', u'extraversion', u'nueroticism', u'openess_to_experience']
y_col = 'Salary'

df_exp = df[X_col + [y_col]]

In [ ]:
df_exp.info()

In [ ]:
##Domain has a few missing values, many models can't handle missing values. For the sake of similicty we will use frame with
## no missing values. We may lose some rows (the ones that have missing values)
df_exp.dropna(subset=X_col, inplace=True)

In [ ]:
##  We lost a few rows, but it's okay. We could have let go of the domain column too, but we believe it could be important
## in predicting the salary
df_exp.shape

In [ ]:
X = df_exp[X_col]
y = df_exp.Salary

In [ ]:
# How does one row of X look?
X.loc[0]

In [ ]:

# Not all columns have numerical values. Not all models can handle categorical variables directly. For example, LinearRegression,
# requires completely numerical input. So we will need to convert the categorical varibles to some numerical values.
# Gender and Degree need our attention.

# For Gender we will convert m --> 1 and f to --> 0 
from sklearn.preprocessing import LabelEncoder
X.loc[X.Gender == 'm', 'Gender'] = 1
X.loc[X.Gender == 'f', 'Gender'] = 0

print X.Gender.unique()

# For Degree, which has 4 unique values, we will get 4 new columns representing each of the types of Degrees. If a candidate is
# from any one of the degrees then that columns will have the value 1 (like ON) and the rest columns will be 0 (OFF).

print 'Old shape', X.shape

X = pd.get_dummies(X, columns=['Degree'])

print 'New shape', X.shape




In [ ]:
## Check the last four new columns
X.columns

In [ ]:
# How does one row of X look now?
X.loc[0]

In [ ]:
# CHECKPOINT #5 - Linear regression

In [ ]:
from sklearn.linear_model import LinearRegression
mdl = LinearRegression()
mdl = mdl.fit(X, y)
## thats it, you've trained your linear regression

In [ ]:
## These are the coefficients of the model
mdl.coef_

In [ ]:
# CHECKPOINT #6 - Error metrics

In [ ]:
## But how does it do? 
explained_variance = mdl.score(X, y)
print explained_variance
## Note we are predicting the model on the data it  used to learn in the first place. 

In [ ]:
## What score --> 0.15 mean? In sklearn regression models, scores are r^2 values. Which define the percentage of variange 
## explained. Root of the same number gives r, which is the correlation between the predicted and the observed values.
correlation = np.sqrt(explained_variance)
print correlation


In [ ]:
## What are some other metrics for model eval?
from sklearn.metrics import mean_squared_error, mean_absolute_error
mse = mean_squared_error(y, mdl.predict(X))
print 'Mean squared error is', mse
print 'Root Mean squared error is', np.sqrt(mse)
print 'Mean Absolute error is', mean_absolute_error(y, mdl.predict(X))
## These give you a better intuitive sense of how off you are on an average in every prediction.

In [ ]:
## What exactly is this corerlation about?
predicted_values = mdl.predict(X)
plt.scatter(predicted_values, y)
plt.title('Predicted v/s Observed')
plt.plot([0,1000000],[0,1000000], c = 'black', alpha =0.5)

In [ ]:
## A little zoomed in?
plt.scatter(predicted_values, y)
plt.title('Predicted v/s Observed')
plt.plot([0,1000000],[0,1000000], c = 'black', alpha =0.5)
plt.xlim((0,1000000))
plt.ylim((0,1000000))

In [ ]:
# CHECKPOINT #7 - DECISION TREES

In [ ]:
## Let's use a decision tree. 
from sklearn.tree import DecisionTreeRegressor
## Note how each model in sklearn uses the same api ie. fit() and predict()
mdl = DecisionTreeRegressor()
mdl = mdl.fit(X, y)
mdl.score(X, y)

In [ ]:
predicted_values = mdl.predict(X)
plt.scatter(predicted_values, y)
plt.title('Predicted v/s Observed')

plt.plot([0,4000000],[0,4000000], c = 'black', alpha=0.5)

In [ ]:
# What? So the r is 1 !!
# The decision tree was able to preict almost every possible values. Imagine a tree with
#leaves for each data row. That's great! We have a model which predict salary with amazing accuracy. 
#But wait, we just trained the model on that data, it ought to know how to predict. But linearregression didn't give an amazing
#accuracy even on the same data. 
# Our DT, had the flexiblity to train meld itself exactly as per the training samples, while the linear regression has the
#constraint to be a line and so it did the best it could do as a line.
#Nonetheless, we trained our model completely to the dataset. But our motive is to predict the grade for samples the model 
#doesn't know about, but does guesses/predicts based on his past learnings.
# So what we need to do is test our models on unseen data. We will split our current dataset into 2 parts - train and test.

In [ ]:
print 'Current dataset size', X.shape

In [ ]:
# CHECKPOINT 8 - TRAIN AND TEST SETS

In [ ]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
## Get it? What happened? We made 4 sets.
print X_train.shape
print X_test.shape
print y_train.shape
print y_test.shape

In [ ]:
## lets train the dt on train set.
#we train on the train set which is 3001 samples.
mdl = DecisionTreeRegressor()
mdl = mdl.fit(X_train, y_train)
r2 = mdl.score(X_test, y_test)
print 'On training data', mdl.score(X_train, y_train)
print 'On unseen test data', r2
# print np.sqrt(r2)

mse = mean_squared_error(y_test, mdl.predict(X_test))
print 'Mean squared error is', mse
print 'Root Mean squared error is', np.sqrt(mse)
print 'Mean Absolute error is', mean_absolute_error(y_test, mdl.predict(X_test))


In [ ]:
#See. That's how bad your model does. That's what you get for using a complicated/overfit model on train and expecting it
# to work better on test. This was toh extreme example of how bad it does. Let's run a linear regression again. 
mdl = LinearRegression()
mdl = mdl.fit(X_train, y_train)
r2 = mdl.score(X_test, y_test)
print 'On training data', mdl.score(X_train, y_train)
print 'On unseen test data', r2


## What are some other metrics for model eval?

mse = mean_squared_error(y_test, mdl.predict(X_test))
print 'Mean squared error is', mse
print 'Root Mean squared error is', np.sqrt(mse)
print 'Mean Absolute error is', mean_absolute_error(y_test, mdl.predict(X_test))


### Why do you think Linear Regression performed better than the DT and the performance didn't change much from the time we ran it on the whole dataset?

Note: DT could have run better if we tweaked its params. Which we will see in the next session.

In [ ]:
# CHECKPOINT #9 - OTHER ML TECHNIQUES

In [ ]:
######### Play area ########
from sklearn.tree import ExtraTreeRegressor, DecisionTreeRegressor
from sklearn.svm import LinearSVR, SVR
from sklearn.ensemble import AdaBoostRegressor, BaggingRegressor, GradientBoostingRegressor, RandomForestRegressor

# Note: We've imported a few models, why don't you play around with them. Most of them have a tweakable parameters but you 
# can run them with their default values and see how they do. Report their r^2 values and other metrics. See how they change.
# Train the models on X_train and predict on X_test. 

# All models in sklearn use the same api. model.fit(X, y) and then model.score(X, y) or model.predict(X)
# You can do DecisionTreeRegressor? to know about the parameters it takes.

# Uncomment the next line and run this cell. It's an example.
# RandomForestRegressor?


### Session 3 - How to select the best parameters?

In [ ]:
from sklearn.grid_search import GridSearchCV
mdl = DecisionTreeRegressor()
params = mdl.get_params()
params
## these are the params in a DT that are tweakable.

In [ ]:
results_test = []
results_train = []
depths = range(1,40)
for depth in depths:
    mdl = DecisionTreeRegressor(max_depth=depth)
    mdl = mdl.fit(X_train, y_train)
    y_pred = mdl.predict(X_test)
    results_test.append(np.sqrt(mean_squared_error(y_pred, y_test)))
    
    y_pred_train = mdl.predict(X_train)
    results_train.append(np.sqrt(mean_squared_error(y_pred_train, y_train)))

plt.figure(figsize=(8,6))
plt.plot(depths, results_test, label='Performance on Test')
plt.plot(depths, results_train, c='red', label = 'Performance on Train')
plt.ylabel('RMSE')
plt.xlabel('max_depth param in DT')
plt.legend()
plt.ylim((0,400000))
plt.title('DT -  train and test rmse on range of max-depth')

In [ ]:
DecisionTreeRegressor?

In [ ]:
# Max depth of three, longer the tree, the more it can overfit, but a too short a tree can't learn much.
# vis a tree with each node taking a decision. Each node has one feature. So your max is 13 for our case.
# How many samples atleast to have in each leaf, in the case we did overfit, each leaf had one sample, so we fitting so badly.
# It's like for each example, the DT had a route. There was no generalization. The tree had rote learned each example. 
# Which turned out bad for us. 
# We are doing GridSearchCV. Groaan. So much type. 
# A grid search takes all the values we give it and runs a model on ver computation and tells us which does better.
# It will run max_depth 3 with min_sample_leaf 1 and so on and so forth
# But how does it evaluate which one is doing better? We learned how to do it using one test and train. So basically if I had
# you to do it, you would run each model and see how it does on ^test^ and then use the hyperparam combo which does the best
# on test for your model. Fair approach. But didn't we overfit on the test data then? And when real unseen data comes to the model,
# it could end up doing bad? 
# Confusing?
# Well, a normal test - train split works if you have a lot of data. A lot of data automatically generalizes quite a bit.
# But in small data..problems. So..why not use another test and split into three? But then when will you stop?
# Enter Cross Validation. You have 100 datapoints. You divide it into 5 folds. 100/5 = 20. So you have 5 sets of 20 points.
# For each fold, you keep aside the 20, and train the model on 80 and test it on the 20 kept aside. You can do this 5 times, one
# for each fold. Finally, you take the average of each score and you get a cv score for that experiement.
# You do the same for each hyper parameter pair you have. And then you pick the one with the best cv score, resting assured
# that the result is quite generalized.
# So we are going to use GridSearchCV to do the same.
# Giving it the values to run the model on. 
params = {'max_depth' : [3,4,5,6,7,8,9,10],
 'min_samples_leaf':[10,20,30,40,50,100,200]
 }
params

In [ ]:
GridSearchCV?

In [ ]:
#init
mdl_cv = GridSearchCV(mdl,params, cv=10 )

# Here it does the magic
# We don't have to test and train necessarily again, CV already helps generalize.
# But just because we did it for regressor, we will do it. NOTE: We don't have to.
mdl_cv = mdl_cv.fit(X_train, y_train)

#This is what it selected
mdl_cv.best_params_

In [ ]:
#Let's see how it does on the test set. 
mdl_cv.score(X_test, y_test)

## Go back up and see how DT did on test set without any parameter tuning! This is quite an improvement. But its still not
## as good as a Linear Regression; which is okay; but you can try tweaking other parameters etc.

In [ ]:
#### PLaygroud #####
# note: why don't you tweak parameters for other models such as the ones imported before?
# get the parameters first, put values in the grid and then run GridSearch to find the best set of hyperparams.
mdl = RandomForestRegressor()
print mdl.get_params()



In [ ]:
## Choose parameters you want to tweak and give them values.
params = {'n_estimators':[10,100,300], 'max_features':[5,6,7], 'min_samples_leaf':[1,10,20] }
##that's a lot of combinations, this tuning is going to take time
mdl_cv = GridSearchCV(mdl,params, cv=5)
mdl_cv = mdl_cv.fit(X_train, y_train)
mdl_cv.best_params_
mdl_cv.score(X_test, y_test) 

In [ ]:
#### Feature selection
from sklearn.feature_selection import f_regression, SelectKBest

In [ ]:
print X_train.shape
clf = SelectKBest(k=16).fit(X_train, y_train)
X_train_k = clf.transform(X_train)
X_test_k = clf.transform(X_test)

In [ ]:
mdl = LinearRegression()
mdl.fit(X_train_k, y_train)
r2 = mdl.score(X_test_k, y_test)

print 'On training data', mdl.score(X_train_k, y_train)
print 'On unseen test data', r2
# print np.sqrt(r2)

mse = mean_squared_error(y_test, mdl.predict(X_test_k))
print 'Mean squared error is', mse
print 'Root Mean squared error is', np.sqrt(mse)
print 'Mean Absolute error is', mean_absolute_error(y_test, mdl.predict(X_test_k))

In [ ]:
# Previous stats of regression with all features 21.

# On training data 0.150187402806
# On unseen test data 0.161858547543
# Mean squared error is 35714037761.1
# Root Mean squared error is 188981.580481
# Mean Absolute error is 101768.379881

In [ ]:
## 16 feature mdl does bad with train data but slightly better with unseen data than the all-feature regression

In [ ]:
##### play ground ####
## Why don't you try playing around with some other methods such as PCA, ICA? 
from sklearn.feature_selection import SelectPercentile
from sklearn.decomposition import PCA, FastICA
## Use the same fit and transform paradigm.
